# ![Ray Databricks Connector](images/databrick_connector_logo.png)
This user guide walks through the basics of reading and writing data with the Ray and Databricks.

The Ray Databricks connector enables parallel read and write to and from a Databricks SQL endpoint. The connector utilizes the Python DB API 2.0 specification implemented by most databases.

## Initialize ray
Ray will automatically be initialized with defaults when calling any ray or ray dataset methods. To specify configuration, like `loggin_level`, add the below. 

In [1]:
import ray, logging
if not ray.is_initialized():
    ray.init()
ray.logger.setLevel(logging.WARN)

2023-02-21 23:19:15,409	INFO worker.py:1242 -- Using address localhost:9031 set in the environment variable RAY_ADDRESS
find: ‘.git’: No such file or directory
2023-02-21 23:19:15,656	INFO worker.py:1360 -- Connecting to existing Ray cluster at address: 10.0.17.214:9031...
2023-02-21 23:19:15,661	INFO worker.py:1548 -- Connected to Ray cluster. View the dashboard at https://console.anyscale.com/api/v2/sessions/ses_vnmb5jgl4z6q98h61dx25rccju/services?redirect_to=dashboard 
2023-02-21 23:19:15,665	INFO packaging.py:330 -- Pushing file package 'gcs://_ray_pkg_c63fb84d82dc4174c6a23f011c36ecdf.zip' (0.81MiB) to Ray cluster...
2023-02-21 23:19:15,676	INFO packaging.py:343 -- Successfully pushed file package 'gcs://_ray_pkg_c63fb84d82dc4174c6a23f011c36ecdf.zip'.


## Connection properties
The databricks  connection properties need to be provided to the data source upon creation. These properties are documented by the databricks.

Below is an example of loading properties from the environment, and filtering them by the 'DATABRICKS_' prefix.

In [2]:
import os

env_connect_props = {
    key.replace('DATABRICKS_','').lower(): value 
    for key,value in os.environ.items() if 'DATABRICKS_' in key
}

env_connect_props = dict(
    server_hostname='dbc-fa3925fd-9102.cloud.databricks.com',
    access_token='10fda01a-72f1-41c7-a397-90b7b09d6679',
    http_path='/sql/1.0/warehouses/db3be239af982b71'
)

# add db and schema in connect props
# add db and schema in connect props
connect_props = dict(
    catalog = 'samples',
    schema = 'tpch',
    **env_connect_props
)

print('Connection properties:')
print('\n'.join(connect_props.keys()))

Connection properties:
catalog
schema
server_hostname
access_token
http_path


# Reading
Ray will use Databricks Python API to read in parallel into a Ray cluster. The created Ray datasets is composed of PyArrow dataframes that are spread across the Ray cluster to allow for the distributed operations required in machine learning.

![Databricks Read](images/databricks_read.png)

### Read from tables
In order to read an entire table into a a Ray cluster, utilize the Ray data `read_databricks` method. The code below will read in a sample table from a Databricks sample database.

In [3]:
from ray.data import read_databricks

# read the table, limiting to first 1K customers
ds = read_databricks(connect_props, table='customer').limit(1000)

# display the first 3 results
ds.limit(3).to_pandas()

(_get_read_tasks pid=226600) opening database connection
(_get_read_tasks pid=226600) executing on database:
(_get_read_tasks pid=226600)   'SELECT COUNT(*) FROM (customer)'
(_get_read_tasks pid=226600) committing database transaction
(_get_read_tasks pid=226600) closing database connection
(_get_read_tasks pid=226600) opening database connection
(_get_read_tasks pid=226600) executing on database:
(_get_read_tasks pid=226600)   'SELECT * FROM (customer) LIMIT 100'


Read progress:   0%|          | 0/1 [00:00<?, ?it/s]

(_get_read_tasks pid=226600) committing database transaction
(_get_read_tasks pid=226600) closing database connection
(_execute_read_task_split pid=226600) opening database connection
(_execute_read_task_split pid=226600) executing on database:
(_execute_read_task_split pid=226600)   'SELECT * FROM (customer) LIMIT 5515 OFFSET 0'


Read progress: 100%|██████████| 1/1 [00:00<00:00, 1376.08it/s]


,c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsegment,c_comment
0,412445,Customer#000412445,"0QAB3OjYnbP6mA0B,kgf",21,31-421-403-4333,5358.33,BUILDING,arefully blithely regular epi
1,412446,Customer#000412446,"5u8MSbyiC7J,7PuY4Ivaq1JRbTCMKeNVqg",20,30-487-949-7942,9441.59,MACHINERY,sleep according to the fluffily even forges. f...
2,412447,Customer#000412447,HC4ZT62gKPgrjr ceoaZgFOunlUogr7GO,7,17-797-466-6308,7868.75,AUTOMOBILE,aggle blithely among the carefully express excus


### Read with a query
For more control over columns and rows read, as well as joining data from multiple tables, a query can be specified instead of a table name. 

In [4]:
QUERY = 'SELECT C_ACCTBAL, C_MKTSEGMENT FROM CUSTOMER WHERE C_ACCTBAL < 0 LIMIT 1000'

# read the result of the query
ds2 = read_databricks(connect_props, query=QUERY)

# display the first 3 results
ds2.limit(3).to_pandas()

(_execute_read_task_split pid=226600) committing database transaction
(_execute_read_task_split pid=226600) closing database connection


(_get_read_tasks pid=226600) opening database connection
(_get_read_tasks pid=226600) executing on database:
(_get_read_tasks pid=226600)   'SELECT COUNT(*) FROM (SELECT C_ACCTBAL, C_MKTSEGMENT FROM CUSTOMER WHERE C_ACCTBAL < 0 LIMIT 1000)'
(_get_read_tasks pid=226600) committing database transaction
(_get_read_tasks pid=226600) closing database connection
(_get_read_tasks pid=226600) opening database connection
(_get_read_tasks pid=226600) executing on database:
(_get_read_tasks pid=226600)   'SELECT * FROM (SELECT C_ACCTBAL, C_MKTSEGMENT FROM CUSTOMER WHERE C_ACCTBAL < 0 LIMIT 1000) LIMIT 100'
(_get_read_tasks pid=226600)

  0%|          | 0/1 [00:00<?, ?it/s]

 committing database transaction


Read progress:   0%|          | 0/1 [00:00<?, ?it/s]

(_get_read_tasks pid=226600) closing database connection
(_execute_read_task_split pid=226600) opening database connection
(_execute_read_task_split pid=226600) executing on database:
(_execute_read_task_split pid=226600)   'SELECT * FROM (SELECT C_ACCTBAL, C_MKTSEGMENT FROM CUSTOMER WHERE C_ACCTBAL < 0 LIMIT 1000) LIMIT 72 OFFSET 0'
(_execute_read_task_split pid=226600) committing database transaction
(_execute_read_task_split pid=226600) closing database connection


Read progress: 100%|██████████| 1/1 [00:00<00:00, 1173.89it/s]


,C_ACCTBAL,C_MKTSEGMENT
0,-219.53,BUILDING
1,-778.23,AUTOMOBILE
2,-848.16,BUILDING


### Additional read parameters
For reading from Databricks, underlying Python API arguments are also available and can be passed to the underlying execute method.

The code below uses the parameters argument to specify parameterss to be used by Databricks when executing the query.

In [5]:
QUERY = 'SELECT C_ACCTBAL, C_MKTSEGMENT FROM CUSTOMER WHERE C_ACCTBAL > %(balance)i LIMIT 100'

ds3 = read_databricks(connect_props, query=QUERY, parameters={'balance':2.0})
ds3.limit(3).to_pandas()

(_get_read_tasks pid=226600) opening database connection
(_get_read_tasks pid=226600) executing on database:
(_get_read_tasks pid=226600)   'SELECT COUNT(*) FROM (SELECT C_ACCTBAL, C_MKTSEGMENT FROM CUSTOMER WHERE C_ACCTBAL > %(balance)i LIMIT 100)'
(_get_read_tasks pid=226600)   parameters=dict(balance=2.0)
(_get_read_tasks pid=226600) committing database transaction
(_get_read_tasks pid=226600) closing database connection
(_get_read_tasks pid=226600) opening database connection
(_get_read_tasks pid=226600) executing on database:
(_get_read_tasks pid=226600)   'SELECT * FROM (SELECT C_ACCTBAL, C_MKTSEGMENT FROM CUSTOMER WHERE C_ACCTBAL > %(balance)i LIMIT 100) LIMIT 100'
(_get_read_tasks pid=226600)   parameters=dict(balance=2.0)


Read progress:   0%|          | 0/1 [00:00<?, ?it/s]

(_get_read_tasks pid=226600) committing database transaction
(_get_read_tasks pid=226600) closing database connection
(_execute_read_task_split pid=226600) opening database connection
(_execute_read_task_split pid=226600) executing on database:
(_execute_read_task_split pid=226600)   'SELECT * FROM (SELECT C_ACCTBAL, C_MKTSEGMENT FROM CUSTOMER WHERE C_ACCTBAL > %(balance)i LIMIT 100) LIMIT 8 OFFSET 0'
(_execute_read_task_split pid=226600)   parameters=dict(balance=2.0)


Read progress: 100%|██████████| 1/1 [00:00<00:00,  4.55it/s]


(_execute_read_task_split pid=226600) committing database transaction


  0%|          | 0/1 [00:00<?, ?it/s]

(_execute_read_task_split pid=226600) closing database connection

Read progress:   0%|          | 0/1 [00:00<?, ?it/s]

Read progress: 100%|██████████| 1/1 [00:00<00:00, 469.11it/s]


,C_ACCTBAL,C_MKTSEGMENT
0,5358.33,BUILDING
1,9441.59,MACHINERY
2,7868.75,AUTOMOBILE


## Writing
The Ray Databricks connector will use the Databricks driver to write each partition of data in parallel. Each partition of data in the Ray dataset will have a write task that writes in parallel to a cload strorage location. After the 
partitions are written, a table is created using the parquet files. This 
![Databricks write](images/databricks_write.png)

### Write to tables
In order to write a dataset into database table, use the `write_dbapi2` method of the dataset object. Repartition the dataset prior to calling this method in order to set the number of write tasks.

In [6]:
from databricks.sql import connect

write_connect_props = {
    **connect_props, 
    'catalog':'hive_metastore',
    'schema':'default'
}

# create destination table
with connect(**write_connect_props) as con:
    with con.cursor() as cursor:
        cursor.execute('DROP TABLE IF EXISTS customer2')
        #cursor.execute('CREATE TABLE customer2 USING DELTA AS (SELECT * FROM samples.tpch.customer LIMIT 0)')
        

The example below writes the previously read data into a new database table that are created using the Snowflake Python API.

In [8]:
# write the dataset to the table 
ds.write_databricks(
    write_connect_props, 
    table='hive_metastore.default.customer2',
    credential='8b246fc0-6157-42de-8d3e-21fc2dff232b-data-access-config-1676359658624',
    stage_uri='s3://egr-sydney-databricks/stage'
)

# read the new table
ds4 = read_databricks(write_connect_props, table='customer2')
ds4.limit(3).to_pandas()

2023-02-21 23:21:46,031	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[write]
write:   0%|          | 0/1 [00:00<?, ?it/s]

(_map_task pid=226600) staging data to s3://egr-sydney-databricks/stage/7fb6e362-de66-4023-a1b8-77fe7aa4f9f4
(_map_task pid=226600) writing db blocks
(_map_task pid=226600) opening database connection
(_map_task pid=226600) adding writer for partition 0
(_map_task pid=226600) CREATE TABLE IF NOT EXISTS customer2
(_map_task pid=226600) executing on database:
(_map_task pid=226600)   'CREATE TABLE IF NOT EXISTS customer2'
(_map_task pid=226600) committing database transaction
(_map_task pid=226600) closing database connection


write: 100%|██████████| 1/1 [00:01<00:00,  1.07s/it]


opening database connection


2023-02-21 23:21:47,145	INFO client.py:193 -- Successfully opened session b'\x01\xed\xb2\x81\x90\xfb\x1c\xe0\xbc\xe8B{p\x96\xac\xd6'


...

(_get_read_tasks pid=226600) 2023-02-21 23:21:53,696	ERROR database_datasource.py:94 -- error executing :

RayTaskError(ServerOperationError): [36mray::_get_read_tasks()[39m (pid=226600, ip=10.0.17.214)
  File "/mnt/cluster_storage/pypi/lib/python3.10/site-packages/ray/data/read_api.py", line 1795, in _get_read_tasks
    requested_parallelism, min_safe_parallelism = _autodetect_parallelism(
  File "/mnt/cluster_storage/pypi/lib/python3.10/site-packages/ray/data/_internal/util.py", line 113, in _autodetect_parallelism
    mem_size = reader.estimate_inmemory_data_size()
  File "/mnt/cluster_storage/pypi/lib/python3.10/site-packages/ray/data/datasource/database_datasource.py", line 376, in estimate_inmemory_data_size
    if self.num_rows and self.sample is not None:
  File "/home/ray/anaconda3/lib/python3.10/functools.py", line 981, in __get__
    val = self.func(instance)
  File "/mnt/cluster_storage/pypi/lib/python3.10/site-packages/ray/data/datasource/database_datasource.py", line 360, in num_rows
    num_rows = self.connector.query_int(query, **self.query_kwargs)
  File "/mnt/cluster_sto


(_get_read_tasks pid=226600)   'SELECT COUNT(*) FROM (hive_metastore.default.customer2)'
(_get_read_tasks pid=226600) Traceback (most recent call last):


(_get_read_tasks pid=226600)   File "/mnt/cluster_storage/pypi/lib/python3.10/site-packages/ray/data/datasource/database_datasource.py", line 283, in execute
(_get_read_tasks pid=226600)     return self._execute(query, **kwargs)
(_get_read_tasks pid=226600)   File "/mnt/cluster_storage/pypi/lib/python3.10/site-packages/ray/data/datasource/dbapi2_datasource.py", line 82, in _execute
(_get_read_tasks pid=226600)     cursor.execute(q, *query_args, **query_kwargs)
(_get_read_tasks pid=226600)   File "/mnt/cluster_storage/pypi/lib/python3.10/site-packages/databricks/sql/client.py", line 472, in execute
(_get_read_tasks pid=226600)     execute_response = self.thrift_backend.execute_command(
(_get_read_tasks pid=226600)   File "/mnt/cluster_storage/pypi/lib/python3.10/site-packages/databricks/sql/thrift_backend.py", line 833, in execute_command
(_get_read_tasks pid=226600)     return self._handle_execute_response(resp, cursor)
(_get_read_tasks pid=226600)   File "/mnt/cluster_storage/pypi/lib

(_get_read_tasks pid=226600) error executing :
(_get_read_tasks pid=226600)   'SELECT COUNT(*) FROM (hive_metastore.default.customer2)'
(_get_read_tasks pid=226600) rolling back database transaction
(_get_read_tasks pid=226600) databricks does not support rollback.
(_get_read_tasks pid=226600) committing database transaction
(_get_read_tasks pid=226600) closing database connection


## Advanced Usage
If more low level access to the Ray Databricks connector is needed, the underlying `DatabricksConnector` and `DatabricksDatasource` can be used.

### Databricks Connector
The `DatabricksConnector` class holds the connection properties and logic required to establish a connection with Databricks. Internally it calls the native Python dirver API in order to read and write from and to tables in parallel across the cluster. The datasource uses the DB API 2 `execute` and `executemany` methods to enable parallel read and writes of data.

The connector is also a Python context manager, and utilize `with` semantics to define when a connection should be established, db operations commited to the database, and the connection closed. 

The code below will read from a sample table using the connector to manage the connection.

In [ ]:
from ray.data.datasource import DatabricksConnector

# query the number of rows, using the connection context to
# manage transactions
with DatabricksConnector(**connect_props) as connector:
    count = connector.query_int(f'SELECT COUNT(*) FROM customer')

print(count)

Alternatively, you can use `try` blocks with the connector's `open`, `commit` and `close` methods. 

In [ ]:
try:
    connector = DatabricksConnector(**write_connect_props)
    connector.open()
    count = connector.query_int(f'SELECT COUNT(*) FROM customer2')
finally:
    connector.close()
    
print(count)

### Databricks Datasource
The Databricks datasource can be used with the Ray data `read_datasource` and `write_datasource` methods to read and write to databases using the distibuted processing capabilities of Ray data. The datasource uses the DatabricksConnector class internally.

Below is an example of creating the datasource using the previously defined connect properties, and then using it to read and write.

In [ ]:
from ray.data.datasource import DatabricksDatasource
from ray.data import read_datasource

# create a datasource from a connector
datasource = DatabricksDatasource(connector)

# use read_datasource to read
ds = read_datasource(datasource, table='customer2')
ds.limit(3).to_pandas()

### DML and DDL
The connector can also be used for any DDL or DML operations you would normally execute through the DB Native Python API. These operations just pass through to the underlying API. 

The code below will create the objects needed for writing to tables. Note that a commit is issued between the queries so the DDL operation executes prior to the next one that is dependent. An alternative is to use two `with` blocks to define transaction boundaries.

In [ ]:
with connector as con:
    con.query('DROP TABLE IF EXISTS customer2')